# Code to visualize each use 

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import librosa
import librosa.display
import load_toilet_data
import pandas as pa
import plot_data
import plot_us
import sqlite3
import numpy as np
import os



import pylab as pl
from scipy.signal import hilbert
from scipy import signal

## Run code block below to get ID numbers for every human use case of the device

In [ ]:
# get a list of every human use

initial_use_l = 1802
use_l = []

print("ID, weight, duration, stool #, urine #")
conn = sqlite3.connect('data/toilet.db')
sql = "SELECT DISTINCT data_capture_id FROM data where data_capture_id > {}".format(initial_use_l)
cursor = conn.execute(sql)

data_capture_dirs = [ f.name for f in os.scandir('./data/data_frames') if f.is_dir() ]

for row in cursor:
        
    # was there a urination event?
    urine_sql = "SELECT timestamp_ms FROM data_start_stop_time WHERE data_capture_id=%d AND type_id=2" % row
    urine_cursor = conn.execute(urine_sql)
    urine_l = []
    for urine_row in urine_cursor:
        urine_l.append(urine_row[0])
    #endfor
    urine_c = int(len(urine_l)/2)
    
    # was there a stool event?
    stool_sql = "SELECT timestamp_ms FROM data_start_stop_time WHERE data_capture_id=%d AND type_id=3" % row
    stool_cursor = conn.execute(stool_sql)
    stool_l = []
    for stool_row in stool_cursor:
        stool_l.append(stool_row[0])
    #endfor
    stool_c = int(len(stool_l)/2)
    
    # was someone actually sitting on the toilet?
    if (len(urine_l) > 0) or (len(stool_l) > 0):
        
        ##########
        # weight # 
        ##########
        
        seatWeight_l = []
        seatWeight_sql = "SELECT value FROM data WHERE data_capture_id=%d AND sensor_id=2" % row
        seatWeight_cursor = conn.execute(seatWeight_sql)
        for seatWeight_row in seatWeight_cursor:
            seatWeight_l.append(seatWeight_row[0])
        #endfor
        seatWeight_sz = pa.Series(seatWeight_l)
        
        footWeight_l = []
        footWeight_sql = "SELECT value FROM data WHERE data_capture_id=%d AND sensor_id=3" % row
        footWeight_cursor = conn.execute(footWeight_sql)
        for footWeight_row in footWeight_cursor:
            footWeight_l.append(footWeight_row[0])
        #endfor
        footWeight_sz = pa.Series(footWeight_l)
        
        totalWeight_median = (footWeight_sz.median() + seatWeight_sz.median())/1000*1.6

        waterDistance_l = []
        waterDistance_sql = "SELECT value FROM data WHERE data_capture_id=%d AND sensor_id=1" % row
        waterDistance_cursor = conn.execute(waterDistance_sql)
        for waterDistance_row in waterDistance_cursor:
            waterDistance_l.append(waterDistance_row[0])
        #endfor
        waterDistance_sz = pa.Series(waterDistance_l)

        
        ##########
        # length # 
        ##########
        min_t = np.inf
        if len(urine_l) > 0:
            if min(urine_l) < min_t: min_t = min(urine_l)
        #endif
        if len(stool_l) > 0:
            if min(stool_l) < min_t: min_t = min(stool_l)
        #endif
        max_t = 0
        if len(urine_l) > 0:
            if max(urine_l) > max_t: max_t = max(urine_l)
        #endif
        if len(stool_l) > 0:
            if max(stool_l) > max_t: max_t = max(stool_l)
        #endif        
        total_t = (max_t-min_t)/1000
        
        # Get files in directory
        files_in_dir = []
        if "data_capture_{}".format(row[0]) in data_capture_dirs:
            files_in_dir = [ f.name for f in os.scandir("./data/data_frames/data_capture_{}".format(row[0])) if f.is_file() ]
        
        # Get file sizes for files in directory
        file_sizes = {}
        if len(files_in_dir) > 0:
            for f in files_in_dir:
                file_sizes[f] = os.stat('./data/data_frames/data_capture_{}/{}'.format(row[0], f)).st_size
        
        # Filters: Do radar, audio, thermal data files exist and are they non-zero?  
        # Is there foot scale data, toilet seat data, ultrasonic data?
        # Is median seat weight greater than threshold value (ex. 3kg)? 
        # Was the ultrasonic sensor working (median greater than threshold and less than a threshold)?
        if "radar_data.txt" in files_in_dir and \
           "audio_data.wav" in files_in_dir and \
           "thermal_data.txt" in files_in_dir and \
           file_sizes["radar_data.txt"] > 200 and \
           file_sizes["audio_data.wav"] > 200 and \
           file_sizes["thermal_data.txt"] > 200 and \
           seatWeight_sz.size > 0 and \
           seatWeight_sz.median() > 3  and \
           footWeight_sz.size > 0 and \
           footWeight_sz.median() > 3 and \
           waterDistance_sz.size > 0 and \
           waterDistance_sz.median() > .08 and \
           waterDistance_sz.median() < 2:    
            use_l.append(row[0])
            print("{:d}, {:f}, {:f}, {:d}, {:d}".format(row[0],totalWeight_median,total_t,stool_c,urine_c))

    #endif
#endfor


## Visualize each use case

In [ ]:
#use_l=[1903]
fig_path = '/home/stephen/python/dl_data_downloader/Data/figs'
for use_i in use_l:
    if use_i > 900:
        print("Visualizing %d" % use_i)
        PlotUse(use_i,fig_path)
    #endif
#endif

## Plotting Methods

In [ ]:
def PlotUse(use_i,fig_path):

    data_d = {}
    data_d[1] = GetSensor(use_i,1) # vertical ultrasound
    data_d[2] = GetSensor(use_i,2) # seat scale
    data_d[3] = GetSensor(use_i,3) # foot scale
    thermal_mi = GetThermal2(use_i)  # thermal camera
    radar_df = GetRadar(use_i)  # radar
    
    t0 = data_d[1][0][0]
    data_d['urineButton'] = GetEvent(use_i,2)
    data_d['stoolButton'] = GetEvent(use_i,3)
    stool_l = [float(i-t0)/1000 for i in data_d['stoolButton']]
    urine_l = [float(i-t0)/1000 for i in data_d['urineButton']]

    event_start_t = np.inf
    event_end_t = 0.
    if len(stool_l) > 0:
        event_start_t =  min(event_start_t,min(stool_l))
        event_end_t = max(event_end_t,max(stool_l))
    #endif

    if len(urine_l) > 0:
        event_start_t = min(event_start_t,min(urine_l))
        event_end_t = max(event_end_t,max(urine_l))
    #endif

    clean1_sz, clean2_sz = cleanSensors(data_d[2][0],data_d[2][1],data_d[3][0],data_d[3][1])

    if use_i < 945:
        fig, (ax1,ax2,ax3,ax4,ax7,ax8,ax9,ax10,ax11,ax12) = plt.subplots(10, sharex=True, figsize=(30,20))
    else: # for now, just assume we're only looking at samples that have audio files.
        fig, (ax1,ax2,ax3,ax4,ax7,ax8,ax9,ax10,ax11,ax12,ax13,ax14) = plt.subplots(12, sharex=True, figsize=(30,20))
    #endif
    
    fig.tight_layout(pad=3.0)
    
    footSeat_space_n = 4
    
    axi = ax1
    seatScale_sz = clean1_sz/1000
    x_ix = (seatScale_sz.index-t0)/1000 #
    axi.plot(x_ix,seatScale_sz,linewidth=3) #
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    
    axi.set_ylabel('seat [kg]')
    axi.grid()
    event_b = (x_ix > event_start_t) & (x_ix < event_end_t) # 
    event_scale_sz = seatScale_sz[event_b]
    axi.set_ylim(seatScale_sz.median()-footSeat_space_n,seatScale_sz.median()+footSeat_space_n)
    #axi.set_ylim(30,90)

    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor

    axi = ax2
    footScale_sz = clean2_sz/1000
    
    x_ix = (footScale_sz.index-t0)/1000 #
    axi.plot(x_ix,footScale_sz,linewidth=3) #
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('footrest [kg]')# 
    axi.grid() #
    event_b = (x_ix > event_start_t) & (x_ix < event_end_t) #   
    
    event_scale_sz = footScale_sz[event_b]
    axi.set_ylim(footScale_sz.median()-footSeat_space_n,footScale_sz.median()+footSeat_space_n)
    #axi.set_ylim(0,90)

    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor

    axi = ax3
    sumScale_sz = seatScale_sz  + footScale_sz
    
    x_ix = (sumScale_sz.index-t0)/1000 #
    axi.plot(x_ix,sumScale_sz,linewidth=3) #
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('sum_scale [kg]')# 
    event_b = (x_ix > event_start_t) & (x_ix < event_end_t) # 
    
    event_scale_sz = sumScale_sz[event_b]
    axi.set_ylim(sumScale_sz.median()-3,sumScale_sz.median()+3)
    #axi.set_ylim(75,95)

    axi.grid()
    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,500,500,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor

    axi = ax4
    data_sz = pa.Series([i*100 for i in data_d[1][1]])

    x_ix = [(i-t0)/1000 for i in data_d[1][0]] #
    axi.plot(x_ix,data_sz,linewidth=3) #
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('water distance [cm]')# 
    axi.grid() #
    
    axi.set_ylim(data_sz.median()-1,data_sz.median()+1)
    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,20,20,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,20,20,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor
    x_ax_lim = axi.get_xlim()
 
    axi = ax7
    t0_df = thermal_mi.loc[thermal_mi.index[0][0]]
    diff_d = {}
    for t, df in thermal_mi.groupby(level=0):
        diff_d[t] = ((t0_df - thermal_mi.loc[t])**2).sum().sum()
    #endfor
    
    diff_sz = pa.Series(diff_d)
    x_ix = (diff_sz.index-t0)/1000 #
    axi.plot(x_ix,diff_sz,linewidth=3) #
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('infrared camera')#  
    
    axi.grid()
    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,8000,8000,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,8000,8000,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor
    axi.set_xlim(x_ax_lim)
    
    # plot thermal camera data
    axi = ax8
    row_df = thermal_mi.groupby(level=0).mean()
    
    xval = (row_df.index - t0)/1000
    yval = range(8)[::-1]
    xx, yy = np.meshgrid(xval, yval)
    axi.pcolormesh(xx,yy,row_df.T,edgecolors='face')
    axi.set_ylabel('row_infrared')
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)

    axi = ax9
    column_df = thermal_mi.mean(1).unstack(level=1)
    xval = (column_df.index - t0)/1000
    yval = range(8)[::-1]
    xx, yy = np.meshgrid(xval, yval)
    axi.pcolormesh(xx,yy,column_df.T,edgecolors='face')
    axi.set_ylabel('column_infrared')
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)

    # plot radar data
    floor_i = 50
    ceil_i = 200
    axi = ax10
    x0 = (radar_df.columns[0]-t0)/1000
    x1 = (radar_df.columns[-1]-t0)/1000
    axi.imshow(radar_df.iloc[::-1],aspect='auto',extent=[x0,x1,radar_df.index[0],radar_df.index[-1]])
    axi.plot([x0,x1],[floor_i,floor_i],'r-')
    axi.plot([x0,x1],[ceil_i,ceil_i],'r-')
    axi.set_ylabel('radar')
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    
    axi = ax11
    area_d = {}
    for i in radar_df.columns:
        sq_sz = (radar_df[i])**2
        area_d[i] = sum(sq_sz.iloc[floor_i:ceil_i])
    #endfor
    area_sz = pa.Series(area_d)
    x_ix = (area_sz.index-t0)/1000 #
    axi.plot(x_ix,area_sz,linewidth=3) #
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('radar sum')
    axi.set_ylim([0,5e9])
    axi.grid()
    for i in range(int(len(stool_l)/2)):
        x = [stool_l[2*i],stool_l[2*i],stool_l[2*i+1],stool_l[2*i+1]]
        y = [0,5e9,5e9,0]
        axi.fill(x,y, 'red', alpha=0.5)
    #endfor
    for i in range(int(len(urine_l)/2)):
        x = [urine_l[2*i],urine_l[2*i],urine_l[2*i+1],urine_l[2*i+1]]
        y = [0,5e9,5e9,0]
        axi.fill(x,y, 'gold', alpha=0.8)
    #endfor
        
          
    if use_i < 945:
        axi = ax12
        S_DB, fs, hop_len = GetAudio1(use_i)
        ax = librosa.display.specshow(S_DB, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel', ax=axi)
        axi.set_ylabel('back mic')
    elif use_i < 1753: 
        axi = ax12
        S_DB, fs, hop_len = GetAudio2(use_i,"back")
        ax = librosa.display.specshow(S_DB, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel', ax=axi)
        axi.set_ylabel('back mic')
        
        axi = ax13
        S_DB, fs, hop_len = GetAudio2(use_i,"front")
        ax = librosa.display.specshow(S_DB, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel', ax=axi)
        axi.set_ylabel('front mic')
    else:
        axi = ax12
        S_DB_front, S_DB_back, fs, hop_len = GetAudio3(use_i)
        ax = librosa.display.specshow(S_DB_front, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel', ax=axi)
        axi.set_ylabel('front mic')
        axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
        
        axi = ax13
        ax = librosa.display.specshow(S_DB_back, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel', ax=axi)
        axi.set_ylabel('back mic')
        axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
        
    
    
    
    
    
    # Time delay beam forming with stereo microphones
    use_dn = "/home/stephen/python/dl_data_downloader/Data/data/data_frames/data_capture_" + str(use_i)
    back_wav_fn = use_dn + "/back_audio_data.wav"
    front_wav_fn = use_dn + "/front_audio_data.wav"
    sampleRate_n = 48000
    x_back, fs = librosa.load(back_wav_fn,sr=sampleRate_n)
    x_front, fs = librosa.load(front_wav_fn,sr=sampleRate_n)

    def frame(data, window_length, hop_length):
      """Convert array into a sequence of successive possibly overlapping frames.
      An n-dimensional array of shape (num_samples, ...) is converted into an
      (n+1)-D array of shape (num_frames, window_length, ...), where each frame
      starts hop_length points after the preceding one.
      This is accomplished using stride_tricks, so the original data is not
      copied.  However, there is no zero-padding, so any incomplete frames at the
      end are not included.
      Args:
        data: np.array of dimension N >= 1.
        window_length: Number of samples in each frame.
        hop_length: Advance (in samples) between each window.
      Returns:
        (N+1)-D np.array with as many rows as there are complete frames that can be
        extracted.
      """
      num_samples = data.shape[0]
      num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
      shape = (num_frames, window_length) + data.shape[1:]
      strides = (data.strides[0] * hop_length,) + data.strides
      return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)
    
    
    EXAMPLE_WINDOW_SECONDS = .1  
    EXAMPLE_HOP_SECONDS = EXAMPLE_WINDOW_SECONDS     # overlap
    window_length = int(round(EXAMPLE_WINDOW_SECONDS * sampleRate_n))
    hop_length = int(round(EXAMPLE_HOP_SECONDS * sampleRate_n))

    back_frames = frame(x_back,window_length,hop_length)
    front_frames = frame(x_front,window_length,hop_length)

    frame_n = back_frames.shape[0]

    delay_d = {}
    delay2_d = {}

    for i in range(frame_n):

        back_frame = back_frames[i,:]
        back_pad = np.concatenate((back_frame,np.zeros(back_frame.shape)),axis=0)

        front_frame = front_frames[i,:]
        front_pad = np.concatenate((front_frame,np.zeros(front_frame.shape)),axis=0)

        corr_cross = np.fft.ifft(np.fft.fft(front_pad)*np.fft.fft(back_pad[::-1]))
        corr_mag = np.abs(corr_cross)
        corr_argmax = np.argmax(corr_mag)

        delay_d[i] = corr_argmax

    #endfor

    delay_sz = pa.Series(delay_d)

    delay_x = delay_sz.index*EXAMPLE_WINDOW_SECONDS

    # Apply envelope
    window_size= 3
    filt_sz = pa.Series(signal.medfilt(delay_sz, window_size))
    #analytic_signal = hilbert(filt_sz)
    #env_sz = pa.Series(np.abs(analytic_signal))
    #window_size = 1
    #filt_sz = pa.Series(signal.medfilt(env_sz, window_size))
    
    filt_sz = pa.DataFrame(filt_sz).rolling(window=30).mean()
    
    axi=ax14
    pl.plot(delay_x, filt_sz)
    pl.grid()
    pl.xticks(np.arange(min(delay_x), max(delay_x)+1, 5.0),rotation=90);
    axi.plot(delay_x, filt_sz,linewidth=3) 
    axi.xaxis.set_major_formatter(librosa.display.TimeFormatter()) # 
    axi.tick_params(axis='x', labelbottom=True, labelrotation=90)
    axi.set_ylabel('mic time delay')# 
    axi.grid() #
    
    
    
    
        
    #endif
    
    axi.xaxis.set_major_locator(plt.MaxNLocator(144))
    plt.xticks(rotation=90)
    ax1.set_title('{}: {}'.format(use_i,round(sumScale_sz.median(),2)),fontsize=24)
    plt.savefig(fig_path + '/' + str(use_i) + '.pdf')
    
#enddef


In [ ]:
def GetAudio3(use_i):
    sampleRate_n = 48000
    from scipy.io import wavfile
    wav_fn  = 'data/data_frames/data_capture_{}/'.format(use_i) + 'audio_data.wav'
    wav_front_fn = 'data/data_frames/data_capture_{}/'.format(use_i) + 'front_audio_data.wav'
    wav_back_fn = 'data/data_frames/data_capture_{}/'.format(use_i) + 'back_audio_data.wav'
    fs, data = wavfile.read(wav_fn) # reading the file
    wavfile.write(wav_front_fn, fs, data[:, 0]) # saving first column which corresponds to channel 1
    wavfile.write(wav_back_fn, fs, data[:, 1])  # saving first column which corresponds to channel 1

    n_fft = 2048
    hop_length = 512
    n_mels = 128
    
    x_front, fs = librosa.load(wav_front_fn,sr=sampleRate_n)
    
    S_front = librosa.feature.melspectrogram(x_front, sr=sampleRate_n, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB_front = librosa.power_to_db(S_front, ref=np.max)
    
    x_back, fs = librosa.load(wav_back_fn,sr=sampleRate_n)
    
    S_back = librosa.feature.melspectrogram(x_back, sr=sampleRate_n, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB_back = librosa.power_to_db(S_back, ref=np.max)

    
    return S_DB_front, S_DB_back, fs, hop_length


In [ ]:
def GetAudio2(use_i,pos_s):
    sampleRate_n = 44100
    wav_fn  = 'data/data_frames/data_capture_{}/'.format(use_i) + pos_s + '_audio_data.wav'
    x, fs = librosa.load(wav_fn,sr=sampleRate_n)

    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(x, sr=sampleRate_n, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB, fs, hop_length

In [ ]:
def GetAudio1(use_i):
    sampleRate_n = 96000
    wav_fn  = 'data/data_frames/data_capture_{}/audio_data.wav'.format(use_i)
    x, fs = librosa.load(wav_fn,sr=sampleRate_n)

    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(x, sr=sampleRate_n, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB, fs, hop_length

In [ ]:
def GetRadar(use_i):

    data_fn = 'data/data_frames/data_capture_{}/radar_data.txt'.format(use_i)
    data_f = open(data_fn,'rt')
    line_s = data_f.read()
    data_l = eval(line_s)

    # save array of images
    t0_sz = pa.Series(data_l[0]['data'])
    data_d = {}
    for j in range(len(data_l)):
        t = data_l[j]['timestamp_ms']
        j_sz = pa.Series(data_l[j]['data'][0])
        data_d[t] = j_sz
    #endfor
    data_df = pa.DataFrame(data_d)
    return data_df

In [ ]:
def GetThermal2(use_i):

    # read in image text file
    thermal_fn = 'data/data_frames/data_capture_{}/thermal_data.txt'.format(use_i)
    thermal_f = open(thermal_fn,'rt')
    line_s = thermal_f.read()
    thermal_l = eval(line_s)

    # save array of images
    t0_sz = pa.Series(thermal_l[0]['data'])
    thermal_d = {}
    for j in range(len(thermal_l)):
        t = thermal_l[j]['timestamp_ms']
        j_df = pa.DataFrame(pa.Series(thermal_l[j]['data']).values.reshape(8,8).T)
        thermal_d[t] = j_df
    #endfor
    thermal_mi = pa.concat(thermal_d.values(),keys=thermal_d.keys())
    
    return thermal_mi

In [ ]:
def cleanSensors(sensor1_t_l,sensor1_y_l,sensor2_t_l,sensor2_y_l):

    # get min / max of time-series
    #sensor1_t_l = data_d[1][0]
    #sensor2_t_l = data_d[2][0]
    #sensor1_y_l = data_d[1][1]
    #sensor2_y_l = data_d[2][1]
    min_t = min(min(sensor1_t_l),min(sensor2_t_l))
    max_t = max(max(sensor1_t_l),max(sensor2_t_l))

    # setup partitions
    step_t = 500
    min_floor_t = int(np.floor(min_t/step_t)*step_t)
    max_ceil_t = int(np.ceil(max_t/step_t)*step_t)
    
    step1_d = {}
    step2_d = {}
    for i in range(min_floor_t,max_ceil_t+step_t,step_t):
        step1_d[i] = []
        step2_d[i] = []
    #endfor

    # step through both and assign values to each partition
    for i in range(len(sensor1_t_l)):
        interval_t = int(np.floor(sensor1_t_l[i]/step_t)*step_t)
        step1_d[interval_t].append(sensor1_y_l[i])
    #endfor
    for i in range(len(sensor2_t_l)):
        interval_t = int(np.floor(sensor2_t_l[i]/step_t)*step_t)
        step2_d[interval_t].append(sensor2_y_l[i])
    #endfor

    # step through each partition and either take averages or set to nan
    clean1_d = {}
    for i in step1_d.keys():
        if(len(step1_d[i]) > 0):
            clean1_d[i] = np.mean(step1_d[i])
    #endfor
    clean1_sz = pa.Series(clean1_d)

    clean2_d = {}
    for i in step2_d.keys():
        if(len(step2_d[i]) > 0):
            clean2_d[i] = np.mean(step2_d[i])
    #endfor
    clean2_sz = pa.Series(clean2_d)
    
    return clean1_sz, clean2_sz

In [ ]:
def GetSensor(use_i,sensor_i):
    sql_s = "SELECT timestamp_ms, value FROM data WHERE data_capture_id={} AND sensor_id={}".format(use_i,sensor_i)
    conn = sqlite3.connect('data/toilet.db')
    cursor = conn.execute(sql_s)
    time_measurements = []
    distance_measurements = []
    for row in cursor:
        time_measurements.append(row[0])
        distance_measurements.append(row[1])
    #endfor
    data_t = (time_measurements,distance_measurements)
    return data_t
#enddef

In [ ]:
def GetEvent(use_i,sensor_i):
    # was there a urination event?
    conn = sqlite3.connect('data/toilet.db')
    sql_s = "SELECT timestamp_ms FROM data_start_stop_time WHERE data_capture_id={} AND type_id={}".format(use_i,sensor_i)
    cursor = conn.execute(sql_s)
    time_l = []
    for row in cursor:
        time_l.append(row[0])
    #endfor
    return time_l
#endfor